# This notebook produces simulations for NoSREx:

1. TB Scatterplot (figure 6c)
2. TB Incidence angle dependency, high resolution (figure 10)
3. TB Incidence angle dependency (10 GHz), low resolution (figure 11)
4. Backscatter angle dependency (figure 12 + figures in appendix)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib import gridspec


# local import
from smrt import make_snowpack, make_model, sensor, make_soil, make_atmosphere
from smrt.substrate.reflector_backscatter import make_reflector
from smrt.core.globalconstants import DENSITY_OF_ICE, FREEZING_POINT
from smrt.utils import dB, invdB

from common_functions import symmetrize_microstructure, microstructure_colour_list
from common_functions import microstructure_short_labels, me, rmse, microstructure_symbols

## Useful functions

In [ ]:
# Reduce number of layers
# Have 320 layers: low resolution=16
def low_res(data, nlayers=16, thickness=False):
    
    n_mean = len(data) / nlayers
    if n_mean.is_integer() is False:
        print ('number of layers must be factor of ', len(data))
    elif thickness == True:
        return np.sum(data.reshape(-1,int(n_mean)), axis=1)
    else:
        return np.mean(data.reshape(-1,int(n_mean)), axis=1)

# From https://gist.github.com/zertrin/4663169
def datetime2matlabdn(dt):
    ord = dt.toordinal()
    mdn = dt + timedelta(days = 366)
    frac = (dt-datetime(dt.year,dt.month,dt.day,0,0,0)).seconds / (24.0 * 60.0 * 60.0)
    return mdn.toordinal() + frac

# https://stackoverflow.com/questions/32237862/find-the-closest-date-to-a-given-date
def nearest_ind(items, pivot):
    time_diff = np.abs([date - pivot for date in items])
    return np.nanargmin(time_diff)


def get_sky_tb(frequency):
    # Retrieves sky brightness temperature
    lookup = str(np.round(frequency / 1e9,2)).split('.0')[0]
    return 0.5 *(sky_df[lookup+'V'] + sky_df[lookup+'H'])

def plot_angular_passive(f, ax):
    # V pol
    ax.plot(theta, dfobs.sel(polarization='V', frequency=f).to_array().squeeze(), 'k^-', label='V-obs', markersize=12)
    [ax.plot(theta, result.TbV(frequency=f, microstructure=m), marker=microstructure_symbols()[m], c=microstructure_colour_list()[m],
         label=('V-'+ microstructure_short_labels()[m])) for m in mlist]
    # H pol
    ax.plot(theta, dfobs.sel(polarization='H', frequency=f).to_array().squeeze(), 'kH-', label='H-obs', alpha=alf, markersize=12)
    [ax.plot(theta, result.TbH(frequency=f, microstructure=m), marker=microstructure_symbols()[m], c=microstructure_colour_list()[m],
         label=('H-'+ microstructure_short_labels()[m]), alpha=alf, linestyle='--') for m in mlist]
    ax.set_ylim([197, 270])
    
# Calculate layer heights for plotting
def calc_layer_heights(thickness):
    return [-thickness[0]/2 - n * thickness[0] for n in range(len(thickness))]

def plot_active(f, ax):
    ax.errorbar(theta, adf.sel(frequency=f, polarization='VV').to_array().squeeze(), marker='*', color='k', linestyle='-', yerr=1, markersize=12, label='VV-obs')
    [ax.plot(theta, dB(result_active.sigmaVV(frequency=f, microstructure=m)), c=microstructure_colour_list()[m], 
             marker=microstructure_symbols()[m],
            label=('VV-' + microstructure_short_labels()[m])) for m in snowpack_labels]
    ax.errorbar(theta, adf.sel(frequency=f, polarization='HH').to_array().squeeze(), marker='*', color='k', linestyle='--', yerr=1, markersize=12, 
            label='HH-obs', alpha=alf)

    [ax.plot(theta, dB(result_active.sigmaHH(frequency=f, microstructure=m)), c=microstructure_colour_list()[m], 
             marker=microstructure_symbols()[m],
            label=('HH-' + microstructure_short_labels()[m]), alpha=alf, linestyle='--') for m in snowpack_labels]
    

## Read in data, assign to layers

In [ ]:
# Microstructure dataframe
mdf = symmetrize_microstructure('data/NoSREx3/nosrex3_CCN_acf_parameters_v0.1.csv')

In [ ]:
# Read in total depth, distribute layer thicknesses evenly between parameter values
# NB ignores sample gaps
automated_data_df = pd.read_csv('data/NoSREx3/NOSREX_automated_in_situ_2011_2012.csv')

# Get time of profile
profiledate = datetime.strptime('01/03/2012', '%d/%m/%Y').replace(hour=14).strftime('%Y/%m/%d %H:%M:%S')

# Get snow depth at time of profile
# Use open area acoustic sensor (SHM1mean). An alternative is AVG_SNOW_AWS: at the AWS
snowdepth = automated_data_df[automated_data_df['OBSTIME_START'] == profiledate]['SHM1mean'].values * 1e-2

# Number of snow layers
nlayers = len(mdf)

# Set up parameter arrays
thickness = np.repeat([snowdepth / nlayers], nlayers)
# Interpolate temperature between surface and ground
surface_temp = min(automated_data_df[automated_data_df['OBSTIME_START'] == profiledate]['TTM1mean'].values 
                   + FREEZING_POINT, FREEZING_POINT)  # This is air temperature
# Take mean of 2 soil temperature observations
B = automated_data_df[automated_data_df['OBSTIME_START']==profiledate]['TSMB2mean']
C = automated_data_df[automated_data_df['OBSTIME_START']==profiledate]['TSMC2mean']
soil_temperature = min(273.15, (B + C).values / 2 + FREEZING_POINT) # limit lower snow layer temperature to 273.15K
# Make profile temperatures
temperature = np.linspace(surface_temp, soil_temperature, nlayers)

In [ ]:
# Read in TB observations
dfobs = pd.read_csv('data/NoSREx3/NOSREX_sodrad_tb_avgs_2011_2012.csv', parse_dates=['OBSTIME'])
# Convert start date from converted_results
dfobs['OBSTIME'] = pd.to_datetime(dfobs['OBSTIME'])  # Make sure in right format
# Create boolean mask to extract only data from snowpit date
mask = (dfobs['OBSTIME'] == profiledate)
dfobs = dfobs.loc[mask]

# Extract only mean values (ignore standard deviation)
dfobs = dfobs.filter(regex='mean').T # Transpose dataframe to make original .csv headers the index

# Get frequency, incidence angle from index names (originally .csv headers)
split = dfobs.index.str.split("_i")
dfobs['polarization'] = [x[0][1:2] for x in split]
dfobs['frequency'] = [int(x[0][2:]) for x in split]
dfobs['theta'] = [int(x[1][:2]) for x in split]

# Set as index and convert to xarray
dfobs = dfobs.set_index(['frequency', 'theta', 'polarization']).to_xarray()

# Relabel frequency into GHz so can reference in the same way as simulations
# Here they are in numerical order i.e. 21, 187, 365, 1065
dfobs['frequency'] = [21e9, 18.7e9, 36.5e9, 10.65e9]
# Put in ascending order of frequency
dfobs = dfobs.sortby('frequency')

In [ ]:
# Sky TB observations
hdrs = ['Date','10.65V', '10.65H', '18.7V', '18.7H', '21V', '21H', '36.5V', '36.5H']
sky_df = pd.read_csv('data/NoSREx3/sodrad_timeseries_90_nosrexiii_sky.txt', header=None, delim_whitespace=True, usecols=range(9))
sky_df.columns = hdrs
profiledate_matlab = datetime2matlabdn(datetime.strptime(profiledate, '%Y/%m/%d %H:%M:%S'))
sky_df = sky_df.iloc[nearest_ind(sky_df['Date'], profiledate_matlab)]
sky_df['Date'] = pd.to_datetime(sky_df['Date']-719529, unit='D')  # Make sure in right format

In [ ]:
# Read in scatterometer data
active_df = pd.read_csv('data/NoSREx3/NOSREX_snowscat_sigma0_avgs_2011_2012_sector1.csv', parse_dates=['time_stamp'])
active_df['time_stamp'] = pd.to_datetime(active_df['time_stamp'])  # Make sure in right format
# # Create boolean mask
mask = (active_df['time_stamp'] == profiledate)
adf = active_df.loc[mask].filter(regex='^sigma0').T  # carat means it has to start with 'sigma0'

# Get frequency, incidence angle from index names (originally .csv headers)
split = adf.index.str.split('_inc')
adf['theta'] = [int(x[1]) for x in split]
adf['polarization'] = [x[0][-2:] for x in split]
adf['frequency'] = [int(x[0][-6:-3]) for x in split]
# Set as index and convert to xarray
adf = adf.set_index(['frequency', 'theta', 'polarization']).to_xarray()

# Relabel frequency into GHz so can reference in the same way as simulations
# Here they are in numerical order i.e. 21, 187, 365, 1065
adf['frequency'] = adf.frequency * 1e8 # Convert to GHz

### Set up simulation configurations

In [ ]:
# Make model
model = make_model("iba","dort")

# Set observation angle
theta = [30, 40, 50, 60]
# Make passive sensor
rad_frequencies = [10.65e9, 18.7e9, 21e9, 36.5e9]
radiometer = sensor.passive(rad_frequencies, theta)
# Make active sensor
scatt_frequencies = [10.2e9, 13.3e9, 16.7e9]
scatterometer = sensor.active(scatt_frequencies, theta)

# Passive Simulations - High Resolution

In [ ]:
# Set up soil
clay=0.01
sand=0.7
drymatter=1300
roughness_rms=2e-2
A = automated_data_df[automated_data_df['OBSTIME_START']==profiledate]['SMMA2mean']
B = automated_data_df[automated_data_df['OBSTIME_START']==profiledate]['SMMB2mean']
moisture = 0.2 # default value
if ((A + B).values > 0) and ((A + B).values < 1):
    moisture = (A + B).values / 2. # Mean of two obs
substrate = make_soil("soil_wegmuller", "dobson85", soil_temperature, moisture=moisture, roughness_rms=roughness_rms,
    clay=clay, sand=sand, drymatter=drymatter)

In [ ]:
# Make snowpacks
snowpack_exp = make_snowpack(thickness, "exponential", density=mdf.density, temperature=temperature, 
                             corr_length=mdf.l_ex, substrate=substrate)
snowpack_shs = make_snowpack(thickness=thickness, microstructure_model="sticky_hard_spheres",
                             temperature=temperature, density=mdf.density, radius=(mdf.d_shs / 2.), 
                             stickiness=mdf.tau, substrate=substrate)
snowpack_ind = make_snowpack(thickness=thickness, microstructure_model="independent_sphere",
                             temperature=temperature, density=mdf.density,radius=(mdf.d_sph / 2.), substrate=substrate)
snowpack_ts = make_snowpack(thickness=thickness, microstructure_model="teubner_strey", temperature=temperature,
                            density=mdf.density, corr_length=mdf.xi_ts, repeat_distance=mdf.domain_ts, substrate=substrate)
snowpack_grf = make_snowpack(thickness=thickness, microstructure_model="gaussian_random_field",
                             temperature=temperature, density=mdf.density, corr_length=mdf.xi_grf, 
                             repeat_distance=mdf.domain_grf, substrate=substrate)
all_snowpacks = [snowpack_exp, snowpack_shs, snowpack_ind, snowpack_ts, snowpack_grf]

# Collect labels for snowpacks: label by microstructure model
snowpack_labels = [all_snowpacks[sp].layers[0].microstructure_model.__name__ for sp in range(len(all_snowpacks))]

#### Add atmosphere to snowpacks

In [ ]:
nadir_tbdown= {f: get_sky_tb(f) for f in rad_frequencies}  # Uncorrected

atmos = make_atmosphere('simple_isotropic_atmosphere', tbdown=nadir_tbdown)

# Add atmosphere to snowpacks
all_snowpacks_with_atmos = [atmos + s for s in all_snowpacks]

#### Run model

In [ ]:
result = model.run(radiometer, all_snowpacks_with_atmos, snowpack_dimension=('microstructure', snowpack_labels))

#### Plot results: figure 6c

In [ ]:
fig, (axv, axh) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(6,12))
plt.rcParams.update({'font.size': 16})

for m in result.Tb().microstructure.values:
    axv.scatter(dfobs.sel(polarization='V').to_array(), result.TbV(microstructure=m), 
               c=microstructure_colour_list()[m], alpha=0.7, edgecolors='none', 
                marker=microstructure_symbols()[m], 
               label='V-' + microstructure_short_labels()[m])
    axh.scatter(dfobs.sel(polarization='H').to_array(), result.TbH(microstructure=m),
               marker=microstructure_symbols()[m], alpha=0.7, c=microstructure_colour_list()[m],
               label='H-' + microstructure_short_labels()[m])


#1-1 line
x = np.linspace(195, 275, 10)
xticks = np.arange(200, 280, 10)
for ax in [axv, axh]:
    ax.plot(x,x,'k--', alpha=0.3)
    ax.set_xticks(xticks)
    ax.set_yticks(xticks)
    ax.set_xlim([195, 275])
    ax.set_ylim([195, 275])

    ax.legend(loc='lower right')
    
plt.xlabel('Observed TB (K)')

plt.tight_layout()
plt.show()

fig.savefig('Fig6c.png')

### Calculate error statistics

In [ ]:
# Mean Error (H, V)
print ('Mean Error (H, V)')

[print (microstructure_short_labels()[m], np.round(me(dfobs.sel(polarization='H').to_array(), result.TbH(microstructure=m)),1), 
        np.round(me(dfobs.sel(polarization='V').to_array(), result.TbV(microstructure=m)),1)) for m in result.Tb().microstructure.values]

In [ ]:
# Root Mean Squared Error (H, V)
print ('Root Mean Squared Error (H, V)')
[print (microstructure_short_labels()[m], np.round(rmse(dfobs.sel(polarization='H').to_array(), result.TbH(microstructure=m)),1), 
        np.round(rmse(dfobs.sel(polarization='V').to_array(), result.TbV(microstructure=m)),1)) for m in result.Tb().microstructure.values]

# Look at angular distribution

Exponential-type models only (plot looks messy otherwise)

In [ ]:
plt.close('all')
plt.rcParams.update({'font.size': 12})
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharex=True)


# Only plot for exponential-type models (including all is too messy)
mlist = ['Exponential', 'TeubnerStrey', 'GaussianRandomField']

# Plot characteristics
alf = 0.7 # alpha

# 10 GHz
plot_angular_passive(10.65e9, ax1)

# 18.7 GHz
plot_angular_passive(18.7e9, ax2)

# 21 GHz
plot_angular_passive(21e9, ax3)

# 36.5 GHz
plot_angular_passive(36.5e9, ax4)

fig.text(0.5, 0.04, 'Incidence angle (degrees)', ha='center', fontsize=12)
fig.text(0.04, 0.5, 'Brightness Temperature (K)', va='center', rotation='vertical', fontsize=12)

# Remove right hand side y axis labels
ax2.set_yticklabels([])
ax4.set_yticklabels([])

# Put frequency labels on graphs
ax1.text(30, 210, '(a) 10.65$\,$GHz', fontsize=12)
ax2.text(30, 210, '(b) 18.7$\,$GHz', fontsize=12)
ax3.text(30, 210, '(c) 21$\,$GHz', fontsize=12)
ax4.text(45, 260, '(d) 36.5$\,$GHz', fontsize=12)

# Legend: but change order
handles, labels = ax4.get_legend_handles_labels()
new_labels = labels[::4] + labels[1::4] + labels[2::4] + labels[3::4]
new_handles = handles[::4] + handles[1::4] + handles[2::4] + handles[3::4]
fig.legend(new_handles, new_labels, 'upper center', bbox_to_anchor=(0.5, 1.01),
          ncol=4, fancybox=True, fontsize=12)
fig.tight_layout(rect=[0.05,0.05,1,0.9])


fig.savefig('Fig10.png')

# Experiment to reduce number of layers

Test 10GHz: this will be most sensitive

In [ ]:
# Low resolution
snowpack_low = make_snowpack(low_res(thickness, thickness=True), "exponential", density=low_res(mdf.density.values), 
                             temperature=low_res(temperature), corr_length=low_res(mdf.l_ex.values), substrate=substrate)
snowpack_low = atmos + snowpack_low
result_low = model.run(radiometer, snowpack_low)

In [ ]:


plt.close('all')
fig = plt.figure(figsize=(6,10))
spec = gridspec.GridSpec(ncols=2, nrows=2, wspace=0.05, hspace=0.2)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[1,:])

# l_ex
ax1.plot(mdf.l_ex * 1e3, calc_layer_heights(thickness), 'k', alpha=0.4, label='n=320')
ax1.plot(low_res(mdf.l_ex.values) * 1e3, calc_layer_heights(low_res(thickness, thickness=True)), 'kx', label='n=16')
ax1.legend(title='l_ex', loc='lower left')
ax1.text(0.1, -0.7, '(a)')
ax1.legend(loc="upper right")
ax1.set_ylabel('Layer depth from snow surface [m]')
ax1.set_xlabel('l$_{ex}$ [mm]')

# density
ax2.plot(mdf.density, calc_layer_heights(thickness),'k', label='HR', alpha=0.4)
ax2.plot(low_res(mdf.density.values), calc_layer_heights(low_res(thickness, thickness=True)), 'kx', label='LR')
ax2.set_xlabel('Density [kg m$^{-3}$]')
ax2.text(100, -0.7, '(b)')
ax2.set_yticklabels([])

# TB
ax3.errorbar(theta, dfobs.sel(polarization='V', frequency=10.65e9).to_array().squeeze(), marker='*', color='k', linestyle='-', yerr=1, label='V-obs', markersize=12)
ax3.errorbar(theta, dfobs.sel(polarization='H', frequency=10.65e9).to_array().squeeze(), marker='*', color='k', linestyle='--', yerr=1, label='H-obs', alpha=0.4, markersize=12)
ax3.plot(theta, result.TbV(frequency=10.65e9, microstructure='Exponential'), 'rX-', label='V (n=320)')
ax3.plot(theta, result.TbH(frequency=10.65e9, microstructure='Exponential'), 'rX--', label='H (n=320)', alpha=0.4)
ax3.plot(theta, result_low.TbV(frequency=10.65e9), 'X-', c='orange', label='V (n=16)')
ax3.plot(theta, result_low.TbH(frequency=10.65e9), 'X--', c='orange', label='H (n=16)', alpha=0.4)
ax3.text(50, 242, '(c)')
ax3.legend(loc='lower left')
ax3.set_xlabel('Incidence angle [degrees]')
ax3.set_ylabel('Brightness Temperature [K]')

fig.savefig('Fig11.png')

# Active simulations

In [ ]:
# Rebuild snowpacks with different substrate
# NB no information on soil contribution to backscatter
soilsigma = make_reflector(temperature=temperature[-1], specular_reflection=0., 
#                           backscattering_coefficient={'VV': 0.1, 'HH': 0.1}) # -10dB
                           backscattering_coefficient={'VV': 0.05, 'HH': 0.05}) # -13dB
#                           backscattering_coefficient={'VV': 0.03, 'HH': 0.03}) # -15dB

# Make snowpacks
a_snowpack_exp = make_snowpack(low_res(thickness, thickness=True), "exponential", 
                             density=low_res(mdf.density.values), temperature=low_res(temperature), 
                             corr_length=low_res(mdf.l_ex.values), substrate=soilsigma)

a_snowpack_shs = make_snowpack(low_res(thickness, thickness=True), microstructure_model="sticky_hard_spheres",
                            density=low_res(mdf.density.values), temperature=low_res(temperature), 
                            radius=(low_res(mdf.d_shs.values) / 2.), 
                            stickiness=low_res(mdf.tau.values), substrate=soilsigma)

a_snowpack_ind = make_snowpack(low_res(thickness, thickness=True), microstructure_model="independent_sphere",
                            density=low_res(mdf.density.values), temperature=low_res(temperature), 
                            radius=(low_res(mdf.d_sph.values) / 2.), substrate=soilsigma)

a_snowpack_ts = make_snowpack(low_res(thickness, thickness=True), microstructure_model="teubner_strey", 
                            density=low_res(mdf.density.values), temperature=low_res(temperature), 
                            corr_length=low_res(mdf.xi_ts.values), repeat_distance=low_res(mdf.domain_ts.values),
                            substrate=soilsigma)

a_snowpack_grf = make_snowpack(low_res(thickness, thickness=True), microstructure_model="gaussian_random_field",
                             density=low_res(mdf.density.values), temperature=low_res(temperature), 
                             corr_length=low_res(mdf.xi_grf.values), repeat_distance=low_res(mdf.domain_grf.values),
                             substrate=soilsigma)

active_snowpacks = [a_snowpack_exp, a_snowpack_shs, a_snowpack_ind, a_snowpack_ts, a_snowpack_grf]
# Collect labels for snowpacks: label by microstructure model
snowpack_labels = [active_snowpacks[sp].layers[0].microstructure_model.__name__ for sp in range(len(active_snowpacks))]

### Run SMRT

In [ ]:
result_active = model.run(scatterometer, active_snowpacks, snowpack_dimension=('microstructure', snowpack_labels))

### Plot active results

In [ ]:
# Plot co-pol
# 10.2 GHz
plt.close()
alf=0.7

fig, ((ax1, ax2, ax3)) = plt.subplots(nrows=1, ncols=3, figsize=(10,5))

plt.rc('axes', titlesize=14)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels

# 10.2 GHz co
plot_active(10.2e9, ax1)
ax1.set_ylabel('Co-Polarized Backscatter (dB)')
ax1.set_ylim(-21, -10)
ax1.text(35, -20, '(a)')
#ax1.set_xticklabels([])
ax1.set_title("10.2 GHz")
ax1.legend(ncol=2, prop={'size': 10})

# 13.3 GHz co
plot_active(13.3e9, ax2)
ax2.text(35, -20, '(b)')
ax2.set_ylim(-21, -10)
#ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.set_xlabel('Incidence angle (degrees)')
ax2.set_title("13.3 GHz")

# 16.7 GHz co
plot_active(16.7e9, ax3)
ax3.text(35, -20, '(c)')
ax3.set_ylim(-21, -10)
#ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax3.set_title("16.7 GHz")

fig.tight_layout()

fig.subplots_adjust(hspace=0.05, wspace=0.05)
fig.savefig('Fig12.png')